In [4]:
import json
import pandas as pd

# Load HateXplain JSON dataset
with open('/kaggle/input/hatexplain-dataset/HateXplain_dataset.json', 'r') as f:
    data = json.load(f)

print(f"Total Data Samples: {len(data)}")

# Extract post text and labels
posts = []
labels = []

for sample in data.values():
    text = sample['post_tokens']  # list of words
    joined_text = ' '.join(text)
    posts.append(joined_text)
    
    # Majority vote label (e.g., 'hate', 'offensive', 'normal')
    majority_label = sample['annotators'][0]['label']
    labels.append(majority_label)

# Convert to DataFrame
df = pd.DataFrame({'text': posts, 'label': labels})
print(df.head())

Total Data Samples: 20148
                                                text       label
0  i dont think im getting my baby them white 9 h...      normal
1  we cannot continue calling ourselves feminists...      normal
2                      nawt yall niggers ignoring me      normal
3  <user> i am bit confused coz chinese ppl can n...  hatespeech
4  this bitch in whataburger eating a burger with...  hatespeech


In [5]:
df['label'].value_counts()

label
normal        8209
hatespeech    5991
offensive     5948
Name: count, dtype: int64

In [6]:
# Install required library if not installed
# !pip install transformers datasets scikit-learn torch pandas

import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# # Load dataset
# df = pd.read_csv('your_dataset.csv')  # Replace with your CSV file name
# print(df.head())

# Encode labels if not already encoded
label_mapping = {'hatespeech': 0, 'offensive': 1, 'normal': 2}
df['label'] = df['label'].map(label_mapping)

# Split into train and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, stratify=df['label'], random_state=42
)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize data
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=300)

# Convert to Hugging Face dataset
train_dataset = Dataset.from_dict({'text': train_texts.tolist(), 'label': train_labels.tolist()})
val_dataset = Dataset.from_dict({'text': val_texts.tolist(), 'label': val_labels.tolist()})

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)

# Remove unnecessary columns
train_dataset = train_dataset.remove_columns(['text'])
val_dataset = val_dataset.remove_columns(['text'])

# Set format for PyTorch
train_dataset.set_format('torch')
val_dataset.set_format('torch')

# Load model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Define metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds, average='weighted'),
        'recall': recall_score(labels, preds, average='weighted'),
        'f1': f1_score(labels, preds, average='weighted')
    }

# Trainer arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to=[]  # disables W&B logging
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/16118 [00:00<?, ? examples/s]

Map:   0%|          | 0/4030 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.834200,0.836931,0.622581,0.623369,0.622581,0.622753
2,0.684800,0.848056,0.632506,0.629310,0.632506,0.630262
3,0.450300,1.015155,0.621340,0.629847,0.621340,0.624087


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=3024, training_loss=0.7008403053990117, metrics={'train_runtime': 1801.5987, 'train_samples_per_second': 26.839, 'train_steps_per_second': 1.679, 'total_flos': 7454640352042800.0, 'train_loss': 0.7008403053990117, 'epoch': 3.0})

In [9]:
# Save the model
trainer.save_model('./hate_speech_classifier')